In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install torch transformers datasets
!pip install accelerate>=0.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset

# Load the SQuAD dataset

squad_dataset=load_dataset("squad")

# Keywords to filter
keywords = ['health',"medicine","medical", 'Anatomy', 'Physiology', 'Pathology', 'Pharmacology', 'Microbiology', 'Immunology', 'Biochemistry', 'Cardiology', 'Dermatology', 'Gastroenterology',
    'Neurology', 'Oncology', 'Radiology', 'Pediatrics', 'Obstetrics', 'Gynecology', 'Orthopedics', 'Ophthalmology', 'Pulmonology', 'Urology',
    'Nephrology', 'Hematology', 'Endocrinology', 'Allergy', 'Psychiatry', 'Genetics', 'Epidemiology', 'Virology', 'Neurosurgery', 'PlasticSurgery',
    'EmergencyMedicine', 'FamilyMedicine', 'InternalMedicine', 'Geriatrics', 'Anesthesiology', 'Dentistry', 'Pharmacy', 'MedicalImaging',
    'MedicalLaboratory', 'Rehabilitation', 'Biotechnology', 'Bioinformatics', 'MedicalEthics', 'PainManagement', 'CriticalCare', 'PalliativeCare',
    'Diabetes', 'Cancer', 'HeartDisease', 'Stroke', 'HIV/AIDS', 'Alzheimer\'sDisease', 'Arthritis', 'Asthma', 'Obesity', 'Depression', 'Schizophrenia',
    'Autism', 'Epilepsy', 'Parkinson\'sDisease', 'Tuberculosis', 'Malaria', 'Hepatitis', 'DiabetesMellitus', 'Hypertension', 'Influenza', 'Ebola',
    'Antibiotics', 'Vaccines', 'MedicalDevices', 'ClinicalTrials', 'ElectronicHealthRecords', 'Telemedicine', 'MedicalEducation', 'MedicalResearch',
    'PublicHealth', 'HealthPolicy', 'GlobalHealth', 'HealthcareFinancing', 'HealthcareAdministration', 'PatientSafety', 'MedicalEthics',
    'HealthInformatics', 'RehabilitationMedicine', 'SportsMedicine', 'OccupationalMedicine', 'ForensicMedicine', 'AlternativeMedicine',
    'TraditionalChineseMedicine', 'Ayurveda', 'Chiropractic', 'Naturopathy', 'Homeopathy', 'HerbalMedicine', 'IntegrativeMedicine',
    'NutritionalMedicine', 'PainManagement', 'SleepMedicine', 'Radiography', 'HealthInsurance', 'PhysicalTherapy', 'OccupationalTherapy',
    'SpeechTherapy', 'MedicalCoding', 'MedicalBilling', 'MedicalTranscription', 'MedicalTerminology', 'HealthcareQuality', 'HealthcareTechnology',
    'HealthDataAnalysis', 'DataPrivacyandSecurity', 'ArtificialIntelligenceinHealthcare', 'MachineLearninginMedicine',
    'NaturalLanguageProcessinginHealthcare', 'BigDatainHealth', 'MedicalRobotics', 'PrecisionMedicine', 'HealthcareAnalytics',
    'HealthcareManagement', 'HealthEconomics', 'MedicalLaw', 'HealthPsychology', 'MentalHealth', 'CommunityHealth', 'Gerontology',
    'HealthPromotion', 'HealthEducation', 'PublicHealthInterventions', 'HealthBehavior', 'HealthCommunication', 'HealthDisparities',
    'EnvironmentalHealth', 'OccupationalHealth', 'IndustrialHygiene', 'InfectiousDiseases', 'GlobalDiseaseOutbreaks', 'MaternalHealth',
    'ChildHealth', 'AdolescentHealth', 'AgingandGeriatricCare', 'HealthcareAccess', 'HealthcareDisparities', 'HealthcareEquity',
    'HealthcareDeliverySystems', 'HealthcareReform', 'HealthcareLegislation', 'HealthTechnologyAssessment', 'HealthRiskAssessment',
    'HealthImpactAssessment', 'HealthMonitoring', 'HealthSurveillance', 'HealthInformatics', 'HealthDataScience', 'HealthDataVisualization',
    'HealthcareDecisionSupportSystems', 'ElectronicMedicalRecords', 'MedicalImageProcessing', 'MedicalSignalProcessing',
    'ClinicalDecisionSupportSystems', 'MedicalDeviceDevelopment', 'HealthcareInformationSystems', 'HealthcareInteroperability',
    'DataMininginHealthcare', 'HealthcarePredictiveAnalytics', 'HealthcareWorkflowOptimization', 'HealthcareSystemIntegration',
    'MobileHealth', 'Telehealth', 'RemotePatientMonitoring', 'DigitalHealth', 'HealthcareApplicationsDevelopment',
    'HealthcareSoftwareEngineering', 'HealthcareUserExperience', 'HealthcareUserInterface', 'PatientEngagement',
    'HealthcareGamification', 'HealthcareMobileApplications', 'HealthcareDataPrivacy', 'HealthcareDataSecurity',
    'Fever', 'Cough', 'Headache', 'SoreThroat', 'RunnyNose', 'Nausea', 'Vomiting', 'Diarrhea', 'Fatigue', 'Rash', 'Allergies',
    'Asthma', 'Bronchitis', 'Pneumonia', 'Influenza', 'CommonCold', 'Sinusitis', 'StomachAche', 'Indigestion', 'Heartburn',
    'Constipation', 'BackPain', 'MusclePain', 'JointPain', 'Arthritis', 'Osteoporosis', 'Anemia', 'Thyroid', 'Menopause',
    'Acne', 'Eczema', 'Psoriasis', 'Migraine', 'Insomnia', 'Depression', 'Anxiety',  'Diabetes', 'Hypertension',
    'Cholesterol', 'HeartDisease', 'Obesity', 'Cancer', 'BreastCancer', 'LungCancer', 'ProstateCancer', 'ColonCancer',
    'SkinCancer', 'Leukemia', 'HIV', 'SexuallyTransmittedInfections', 'UrinaryTractInfection', 'KidneyStones', 'Gallstones',
    'LiverDisease', 'Hepatitis', 'Cirrhosis', 'FoodPoisoning', 'FoodAllergies', 'GlutenIntolerance', 'LactoseIntolerance',
    'Anorexia', 'Bulimia', 'ObstructiveSleepApnea', 'Snoring', 'Osteoarthritis', 'RheumatoidArthritis', 'Gastritis',
    'PepticUlcer', 'Hemorrhoids', 'CarpalTunnelSyndrome', 'Fibromyalgia', 'Endometriosis', 'PremenstrualSyndrome',
    'ErectileDysfunction', 'Infertility', 'Pregnancy', 'Miscarriage', 'Menstruation', 'Contraception', 'BirthControl',
    'PostpartumDepression', 'Breastfeeding', 'ChildhoodVaccinations', 'Autism', 'DownSyndrome', 'Scoliosis', 'Chickenpox',
    'Measles', 'Mumps', 'Rubella', 'WhoopingCough', 'DiaperRash', 'Teething', 'Colic', 'Croup', 'EarInfection', 'StrepThroat',
    'Impetigo', 'Ringworm', 'Pinkeye', 'HeadLice', 'Bedwetting', 'Tonsillitis', 'Conjunctivitis', 'Nosebleed', 'Warts', 'Hives',
    'Sunburn','Fever', 'Cough', 'Headache', 'Sore throat', 'Runny nose', 'Congestion', 'Nausea', 'Vomiting', 'Diarrhea', 'Fatigue',
    'Rash', 'Itching', 'Allergies', 'Asthma', 'Bronchitis', 'Pneumonia', 'Influenza', 'Common cold', 'Sinusitis',
    'Stomach ache', 'Indigestion', 'Heartburn', 'Constipation', 'Diabetes', 'Hypertension', 'High blood pressure', 'Cholesterol',
    'Heart disease', 'Stroke', 'Obesity', 'Arthritis', 'Back pain', 'Joint pain', 'Muscle pain', 'Osteoporosis', 'Gout',
    'Anemia', 'Thyroid', 'Menopause', 'Acne', 'Eczema', 'Psoriasis', 'Dandruff', 'Migraine', 'Insomnia', 'Sleep apnea',
    'Depression', 'Anxiety', 'Bipolar disorder', 'ADHD',
    'Autism spectrum disorder (ASD)', 'Schizophrenia', 'Alzheimer\'s', 'Dementia', 'Parkinson\'s', 'Cancer', 'Breast cancer',
    'Lung cancer', 'Prostate cancer', 'Colon cancer', 'Skin cancer', 'Leukemia', 'HIV/AIDS', 'Sexually transmitted infections (STIs)',
    'Urinary tract infection (UTI)', 'Kidney stones', 'Gallstones', 'Liver disease', 'Hepatitis', 'Cirrhosis', 'Food poisoning',
    'Food allergies', 'Gluten intolerance', 'Lactose intolerance', 'Anorexia', 'Bulimia', 'Obstructive sleep apnea', 'Snoring',
    'Osteoarthritis', 'Rheumatoid arthritis', 'Gastritis', 'Peptic ulcer', 'Hemorrhoids', 'Carpal tunnel syndrome', 'Tennis elbow',
    'Fibromyalgia', 'Endometriosis', 'Polycystic ovary syndrome (PCOS)', 'Erectile dysfunction', 'Premenstrual syndrome (PMS)',
    'Fertility', 'Menstrual cramps', 'Infertility', 'Pregnancy', 'Miscarriage', 'Ectopic pregnancy', 'Menstruation', 'Menopause',
    'Contraception', 'Birth control', 'Morning sickness', 'Prenatal care', 'Postpartum depression', 'Childbirth', 'Breastfeeding',
    'Childhood vaccinations', 'Autism', 'Down syndrome', 'Attention deficit hyperactivity disorder (ADHD)', 'Scoliosis', 'Asthma',
    'Chickenpox', 'Measles', 'Mumps', 'Rubella', 'Whooping cough', 'Hand, foot, and mouth disease', 'Diaper rash', 'Teething',
    'Colic', 'Croup', 'Ear infection', 'Strep throat', 'Fifth disease', 'Impetigo', 'Ringworm', 'Pinkeye', 'Head lice',
    'Bedwetting', 'Thumb sucking', 'Tonsillitis', 'Adenoids', 'Conjunctivitis', 'Nosebleed', 'Warts', 'Hives', 'Sunburn','stress']

s=set(keywords)
keywords=list(s)

# Filter dataset based on keywords
#filtered_dataset = squad_dataset.filter(lambda example: any(keyword in example["question"] for keyword in keywords))
filtered_dataset = squad_dataset.filter(lambda example: any(keyword.lower() in example["question"].lower() for keyword in keywords))
dataset=filtered_dataset


# Load the BioBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForQuestionAnswering.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Tokenize and preprocess the dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

dataset = dataset.map(preprocess_function, batched=True)



Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10570 [00:00<?, ? examples/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1038 [00:00<?, ? examples/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

In [ ]:
# Define the training arguments
training_args = TrainingArguments(output_dir='bruv',

    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_steps=100,
    save_steps=500,

    disable_tqdm=False



)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],

)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,4.030298
2,4.555400,3.669961
3,4.555400,3.204600
4,3.319900,2.994030
5,2.431000,2.931981
6,2.431000,2.996245
7,1.841200,2.909187
8,1.359000,3.042550
9,1.359000,3.287651
10,1.049000,3.265278


TrainOutput(global_step=1950, training_loss=0.9633790761996538, metrics={'train_runtime': 2630.6278, 'train_samples_per_second': 11.837, 'train_steps_per_second': 0.741, 'total_flos': 6102585753569280.0, 'train_loss': 0.9633790761996538, 'epoch': 30.0})

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')
'''

"from google.colab import drive\ndrive.mount('/content/drive')\n"

In [ ]:
output_dir='/content/drive/MyDrive/samcli'

In [ ]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/samcli/tokenizer_config.json',
 '/content/drive/MyDrive/samcli/special_tokens_map.json',
 '/content/drive/MyDrive/samcli/vocab.txt',
 '/content/drive/MyDrive/samcli/added_tokens.json',
 '/content/drive/MyDrive/samcli/tokenizer.json')

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [ ]:
import random
x=random.randrange(0,138)
#print(x)
question = dataset["validation"][89]["question"]
context =  dataset["validation"][89]["context"]

#question="what are the main types of diabetes"
#context = "Diabetes mellitus is a chronic metabolic disorder characterized by high blood sugar levels. It occurs when the body either doesn't produce enough insulin or doesn't effectively use the insulin it produces. There are two main types of diabetes: type 1 and type 2. Type 1 diabetes is an autoimmune condition in which the immune system mistakenly attacks and destroys the insulin-producing cells in the pancreas. Type 2 diabetes is more common and is often associated with lifestyle factors such as obesity, physical inactivity, and poor diet. Common symptoms of diabetes include increased thirst, frequent urination, unexplained weight loss, fatigue, blurred vision, and slow wound healing. Proper management of diabetes involves regular monitoring of blood sugar levels, a healthy diet, regular exercise, and, in some cases, insulin or other medications."
#print(context)
print("Question:",question)

# Tokenize input
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

# Retrieve input IDs and attention masks
input_ids = inputs["input_ids"].tolist()
attention_mask = inputs["attention_mask"].tolist()



Question: What is the minimum distance between a patient's home and the nearest pharmacy that allows a physician in Austria to give out medicine?


In [ ]:
model.eval()

# Make the prediction
with torch.no_grad():
    outputs = model(input_ids=torch.tensor(input_ids), attention_mask=torch.tensor(attention_mask))

# Get the start and end logits from the model's output
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Retrieve the predicted answer span
all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
answer = tokenizer.convert_tokens_to_string(all_tokens[torch.argmax(start_logits) : (torch.argmax(end_logits)+1)])
#answer = tokenizer.convert_tokens_to_string(all_tokens[73 : 103])

#print(torch.argmax(start_logits))
#print(torch.argmax(end_logits)+1)
print("Predicted Answer: [Clinincal Bert]", answer)

Predicted Answer: [Clinincal Bert] 1. 6 kilometres
